In [19]:
import pandas as pd
import sasoptpy as so
import re
import os
import subprocess
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
t0 = time.time()

In [21]:
df = pd.read_csv('../data/input.csv')
gk_data = df[df['Pos'] == 'G'].copy().reset_index()
gk_data.set_index('index', inplace=True)

In [22]:
def find_optimal_goalkeepers(gk_data, budget):
    # Initializing the model
    model = so.Model(name='gk_model')
    players = gk_data.index.tolist()
    
    # Variables
    lineup = model.add_variables(players, name='lineup', vartype=so.binary)
    bench = model.add_variables(players, name='bench', vartype=so.binary)
    
    # Objective
    total_xp = so.expr_sum(lineup[p] * gk_data.loc[p, '3_Pts'] for p in players) + 0.05 * so.expr_sum(bench[p] * gk_data.loc[p, '3_Pts'] for p in players)
    model.set_objective(-total_xp, name='total_xp_obj', sense='N')
    
    # Constraints
    model.add_constraints((lineup[p] + bench[p] <= 1 for p in players), name='lineup_or_bench')
    model.add_constraint(so.expr_sum(lineup[p] for p in players) == 1, name='single_lineup')
    model.add_constraint(so.expr_sum(bench[p] for p in players) == 1, name='single_bench')
    model.add_constraint(so.expr_sum((lineup[p] + bench[p]) * gk_data.loc[p, 'BV'] for p in players) <= budget, name='budget_constraint')
    
    # Solution
    model.export_mps(filename='gk.mps')
    command = 'cbc gk.mps solve solu solution.txt'
    # !{command}
    #os.system(command)
    subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Parse solution
    with open('solution.txt', 'r') as f:
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))

    # Print results
    print("LINEUP")
    for p in players:
        if lineup[p].get_value() > 0.5:
            print(p, gk_data.loc[p])
    
    print("BENCH")
    for p in players:
        if bench[p].get_value() > 0.5:
            print(p, gk_data.loc[p])

In [23]:
find_optimal_goalkeepers(gk_data, 9.5)

NOTE: Initialized model gk_model.
LINEUP
14 Pos               G
ID               15
Name           Raya
BV              5.5
SV              5.5
Team        Arsenal
3_xMins          93
3_Pts          4.07
4_xMins          93
4_Pts          3.55
5_xMins          90
5_Pts          3.33
6_xMins          91
6_Pts          4.36
7_xMins          90
7_Pts           4.3
8_xMins          89
8_Pts          3.73
9_xMins          88
9_Pts          3.42
10_xMins         88
10_Pts         3.48
Elite%         0.09
Name: 14, dtype: object
BENCH
465 Pos                   G
ID                  466
Name             Lumley
BV                  4.0
SV                  4.0
Team        Southampton
3_xMins               2
3_Pts              0.07
4_xMins               2
4_Pts              0.06
5_xMins               2
5_Pts              0.09
6_xMins               2
6_Pts              0.08
7_xMins               3
7_Pts              0.11
8_xMins               4
8_Pts              0.15
9_xMins               4
9_Pts 

In [6]:
t1 = time.time()
print(t1-t0, 'seconds')

0.1166079044342041 seconds
